# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

They want to explore...

What do the conditions mean?

oxy: the groups of mice were treated with oxycodone, which is an opioid


sal: is given to the control groups and is sterile saline

What do the genotypes mean?

SNI: This stands for Spared nerve injury. These mice had a surgery, that induced this chronic nerve pain. 


Sham: Is a control group. These mice did all the same treatment and surgery steps, but the final nerve injury was not introduced, so they should not have the chronic pain. 

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?

Which groups would you compare to each other?

Please also mention which outcome you would expect to see from each comparison.

If I would have been ask to analyze a raw file from this study, I would first check: 
    - which experiments do I have/in which files is which experiments
    - where are the timepoint distributed (is every analysis time point a different table?)
    
I woudld compare all 4 groups which is other like they did in the paper. And then I woudl liek to see the same results they showed.
    

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [10]:
import pandas as pd

metadata = pd.read_excel("conditions_runs_oxy_project.xlsx", index_col="Run")
metadata


,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
Run,,,,,,,
SRR23195505,?,x,NaN,x,NaN,x,NaN
SRR23195506,?,x,NaN,NaN,x,NaN,x
SRR23195507,?,x,NaN,x,NaN,NaN,x
SRR23195508,?,x,NaN,NaN,x,x,NaN
SRR23195509,?,x,NaN,NaN,x,x,NaN
SRR23195510,?,x,NaN,x,NaN,x,NaN
SRR23195511,?,x,NaN,NaN,x,NaN,x
SRR23195512,?,x,NaN,x,NaN,NaN,x
SRR23195513,?,x,NaN,x,NaN,x,NaN


In [ ]:
# How many samples do you have per condition?
numSal = metadata["condition: Sal"].notna().sum()
numOxy = metadata["Condition: Oxy"].notna().sum()
print("Samples for the Sal condition:", numSal)
print("Samples for the Oxy condition:", numOxy)

# How many samples do you have per genotype?
numSni = metadata["Genotype: SNI"].notna().sum()
numSham = metadata["Genotype: Sham"].notna().sum()
print("Samples for the SNI genotype:", numSni)
print("Samples for the Sham genotype:", numSham)

# How often do you have each condition per genotype?
print("\nCondition per genotype:")
for g in genotypes:
    for c in conditions:
        count = metadata[metadata[g].notna() & metadata[c].notna()].shape[0]
        print(f"{g} + {c}: {count}")



Samples for the Sal condition: 8
Samples for the Oxy condition: 8
Samples for the SNI genotype: 8
Samples for the Sham genotype: 8

Condition per genotype:
Genotype: SNI + condition: Sal: 4
Genotype: SNI + Condition: Oxy: 4
Genotype: Sham + condition: Sal: 4
Genotype: Sham + Condition: Oxy: 4


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [13]:
counts = pd.read_csv("base_counts.csv", index_col="Run")
counts

,Bases
Run,
SRR23195505,6922564500
SRR23195506,7859530800
SRR23195507,8063298900
SRR23195508,6927786900
SRR23195509,7003550100
SRR23195510,7377388500
SRR23195511,6456390900
SRR23195512,7462857900
SRR23195513,8099181600


In [16]:
metadata["Bases"] = counts["Bases"]
metadata.sort_values("Bases", ascending=True)

,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Bases
Run,,,,,,,,
SRR23195516,?,x,NaN,NaN,x,x,NaN,6203117700
SRR23195511,?,x,NaN,NaN,x,NaN,x,6456390900
SRR23195517,?,x,NaN,NaN,x,x,NaN,6863840400
SRR23195505,?,x,NaN,x,NaN,x,NaN,6922564500
SRR23195508,?,x,NaN,NaN,x,x,NaN,6927786900
SRR23195519,?,x,NaN,NaN,x,NaN,x,6996050100
SRR23195509,?,x,NaN,NaN,x,x,NaN,7003550100
SRR23195514,?,x,NaN,NaN,x,NaN,x,7226808600
SRR23195510,?,x,NaN,x,NaN,x,NaN,7377388500


In [ ]:
# Prepare csv file


Index(['Patient', 'RNA-seq', 'DNA-seq', 'condition: Sal', 'Condition: Oxy',
       'Genotype: SNI', 'Genotype: Sham', 'Bases'],
      dtype='object')

In [24]:
# Run pipline
!nextflow run nf-core/fetchngs -profile docker --input ids.csv --outdir test

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh: 0: getcwd() failed: No such file or directory
Error occurred during initialization of VM
java.lang.Error: Properties init: Could not determine current working directory.
	at jdk.internal.util.SystemProps$Raw.platformProperties(java.base@17.0.16/Native Method)
	at jdk.internal.util.SystemProps$Raw.<init>(java.base@17.0.16/SystemProps.java:234)
	at jdk.internal.util.SystemProps.initProperties(java.base@17.0.16/SystemProps.java:54)
	at java.lang.System.initPhase1(java.base@17.0.16/System.java:2106)
NOTE: Nextflow needs a Java virtual machine to run. To this end:
 - make sure a `java` command can be found; or
 - manually define the variables JAVA_HOME to point to an existing installation; or
 - install a Java virtual machine, for instance through https://sdk

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.